In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, random_split


transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# Loading dataset
svhn_train_full = datasets.SVHN(root='./data', split='train', download=True, transform=transform)
svhn_test_full = datasets.SVHN(root='./data', split='test', download=True, transform=transform)

subset_size = int(0.25 * len(svhn_train_full))

svhn_train_subset, _ = random_split(svhn_train_full, [subset_size, len(svhn_train_full) - subset_size])
svhn_test_subset, _ = random_split(svhn_test_full, [subset_size, len(svhn_test_full) - subset_size])

models_dict = {
    'AlexNet': models.alexnet(),
    'VGG-11': models.vgg11(),
    'ResNet-18': models.resnet18(),
}

train_loader = DataLoader(svhn_train_subset, batch_size=64, shuffle=True)
test_loader = DataLoader(svhn_test_subset, batch_size=64, shuffle=False)

for model_name, model in models_dict.items():

    if 'resnet' in model_name.lower():
        num_ftrs = model.fc.in_features
        model.fc = nn.Linear(num_ftrs, 10)
    else:
        num_ftrs = model.classifier[-1].in_features
        model.classifier[-1] = nn.Linear(num_ftrs, 10)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.99)  # Increased learning rate for faster training
    num_epochs = 1  # Reduced number of epochs for faster training

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0

        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        print(model_name + " - Epoch " + str(epoch+1) + ", Loss: " + str(running_loss/len(train_loader)))

    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print(model_name + " -test set Accuracy =>: " + "{:.2f}%".format((correct/total)*100))



100%|██████████| 182040794/182040794 [00:09<00:00, 19401034.13it/s]


100%|██████████| 64275384/64275384 [00:06<00:00, 10516921.40it/s]


AlexNet - Epoch 1, Loss: 2.247962195698808
AlexNet -test set Accuracy =>: 19.69%
VGG-11 - Epoch 1, Loss: nan
VGG-11 -test set Accuracy =>: 6.61%
ResNet-18 - Epoch 1, Loss: 2.700976387548945
ResNet-18 -test set Accuracy =>: 19.48%
